In [1]:
interp.repositories() ++= Seq(coursierapi.MavenRepository.of(
"https://jitpack.io"
))

In [2]:
import $ivy. `org.carbonateresearch::picta:0.1`

import $ivy.$                                  

In [3]:
import org.carbonateresearch.picta.render.Html.initNotebook // required to initialize jupyter notebook mode
initNotebook() // stops ugly output

import org.carbonateresearch.picta.render.Html.initNotebook // required to initialize jupyter notebook mode


# 1. Basics

The aim of the Picta library is to be a highly configurable and composable charting library for data exploration. The library takes a grammatical approach to chart construction. 

The following examples are aimed at demonstrating the libraries capabilities, and eventually make constructing charts using this library easy and intuitive.

### Main Components

Picta graphs are constructed in a modular manner. The main components are:

1. `Canvas`: This is is the top-level component for representing the display. A `Canvas` may consist of multiple charts.


2. `Chart`: This is the component that corresponds to an actual chart (i.e. scatter, pie etc).


3. `Series`: This is the data series that is plotted on a `Chart`. `Series` come in 3 types:

    - `XY`:   This is series data for constructing a 2D chart
    - `XYZ`:  This is series data for constructing a 3D chart
    - `Map`:  This is series data that constructs a Map chart
    

4. `ChartLayout`: This configures various parts of how the `Chart` is represented on screen.

`Chart` may occassionally also make use of the `Config` component for further configuration. `Config` specifies whether the plot is responsive.

The following examples in this notebook should provide a number of interactive examples to get used to the Picta library.

### Importing the Library

Importing as below should bring in the most frequently used Components in the library.

In [4]:
import org.carbonateresearch.picta._

### Create Some Dummy Data to Use in the Examples

Here we create some dummy data to be used in the examples for this notebook.

In [5]:
val x = List.range(0, 100).map(x => scala.util.Random.nextDouble() * 50)
val y = List.range(0, 100).map(x => scala.util.Random.nextDouble() * 50)
val z = List.range(0, 100).map(x => scala.util.Random.nextDouble() * 50)

### 2D Plots

`Series` that represent 2D data are represented by `XY` series types.


#### Scatter Plot

An example `Scatter` plot is below. 

Note: This is just one way to call the library functions. Another way that uses function chaining will be shown immediately after this example.

In [6]:
// create the data series using the data from above
// 'asType SCATTER' transforms the series into a scatter chart
// 'drawStyle Markers' specifies the style of the markers the chart should have
// if a name is not provided to the series, Picta generates a random name to help keep track of the series in charts
// multiple series.
val series = XY(x, y) asType SCATTER drawStyle MARKERS

// create a chart by adding to it a data series. We set the title as 'First Chart'
val chart = Chart() addSeries series setTitle "First Chart"

// if we have a single chart, then we do not need to use the Canvas and can simply call chart.plot

chart.plotInline

#### Chaining Functions

Sometimes when we are specifying many options, it is quicker to use the IDE autocomplete and just chain the function calls. Throughout this book both the style above and the function chaining style will be used as and when convenient.

In [7]:
// sometimes we may specify many options. In such cases it is clearer to use function chaining as below:
val series = XY(x, y)
            .asType(SCATTER)
            .drawMarkers  // This is to help leverage the IDE autocompletion for quick scripting

val chart = Chart()
            .addSeries(series)
            .setTitle("First Chart - Function Chaining")
            // by specifying axes we can get rid of the zerolines
            .addAxes(XAxis(zeroline = false), YAxis(zeroline = false)) 

chart.plotInline

#### Bar Chart

Now using the same principles as above, let's create a `Bar` chart:

In [8]:
// First we create some data for the piechart
val x_bar = List("a", "b", "c")
val y_bar = List(10, 20, 300)

// again feed in the data that we want to represent as a Bar chart.
// Notice how we change 'asType SCATTER' to 'asType BAR'
val series = XY(x_bar, y_bar) asType BAR

// we can turn off the scrollzoom using setConfig
val chart = Chart() addSeries series setTitle "Bar Chart" setConfig(false, false)

chart.plotInline

#### Adding Multiple Series on the Same Axis

Sometimes we may want to plot multiple `Series` on the same axis to compare different data series.

Let's plot both of the above on the same pair of axis:

In [9]:
// we import the marker option which lets us specify the marker
import org.carbonateresearch.picta.options.Marker

// these are the various shapes for the markers
import org.carbonateresearch.picta.options.SymbolShape.{CIRCLE, SQUARE_OPEN}

In [10]:
// additional traces can simply be composed with an existing chart and added on
val series1 = XY(x, y) asType BAR setName "Bar"

// lets give the second series a red marker. Again we can 'compose' a marker using smaller components
val marker = Marker() setSymbol SQUARE_OPEN setColor "red"

val series2 = XY(x, y) asType SCATTER setName "Scatter" drawStyle MARKERS setMarker marker

// we not put brackets in the 'addSeries' function to ensure that addSeries picks up the right series'
val chart = Chart() addSeries(series1, series2)

chart.plotInline

### Lets Change The Appearance of The Markers

We can control the appearance of the markers by setting the width

In [11]:
// we can pass in the size of the markers
val sizes = List.range(0, 100).map(x => 10)

// additional traces can simply be composed with an existing chart and added on
val series1 = XY(x, y) asType BAR setName "Bar"

// lets give the second series a red marker. Again we can 'compose' a marker using smaller components
val marker = (
    Marker() 
    // sets the marker shape
    setSymbol CIRCLE
    // sets the fill to red
    setColor "red" 
    // sets the outline to black, with width 2
    setLine(width = 2, "black")
    // set the size of the markers to 10 as per the list we created
    setSize(sizes)
) 

val series2 = XY(x, y) asType SCATTER setName "Scatter" drawStyle MARKERS setMarker marker

// we not put brackets in the 'addSeries' function to ensure that addSeries picks up the right series'
val chart = Chart() addSeries(series1, series2) setConfig(false, false)

chart.plotInline

### Adding Another Axes

Sometimes we may wish to plot multiple series, but on two different Axes. We can do this as follows:

In [12]:
val series1 = XY(x, y) asType SCATTER drawStyle MARKERS

// The following maps the series onto the second Y axis.
val series2 = (
    XY(x, z) 
    asType SCATTER 
    drawStyle MARKERS 
    setAxis YAxis(2)
)
      
val chart = (
    Chart() 
    setTitle "Multiple Axes"
    // the following makes the chart unresponsive
    setConfig(false, false) 
    addSeries(series1, series2) 
    // the following tells the chart how t render the second Y Axis
    addAxes YAxis(position = 2, title = "Second y axis", overlaying = YAxis(), side = RIGHT)
    addAxes YAxis(1, "First Y Axis")
    addAxes XAxis(title = "X Axis")
) 

// this is just for illustration purposes, but we can also do the following
val canvas = Canvas() setChart(0, 0, chart)

canvas.plotInline

### Customizing the Axes

As can be seen from some of the examples above, the axes were not set. While for quick data exploration the above is acceptable, for presentation purposes we would want named axes.

We can do this as follows:

In [13]:
// We construct the axes and set their title
val xaxis = XAxis(title = "x variable")
val yaxis = YAxis(title = "y variable")

// another way to do composition is to just add a bracket around the composition
val chart = (
    Chart()
    setTitle "Chart with Axes"
    addSeries(series1)
    addAxes(xaxis, yaxis)
)

chart.plotInline

### Display options - MultiCharts

`MultiCharts` are a way to plot multiple axes on the same chart. This is seperate to Picta's grid system, which create a grid of subplot `Chart`'s. The next series of examples will make this clearer.

In [14]:
// create the axes
val xaxis1 = XAxis() setTitle "x1"
val yaxis1 = YAxis() // this is not strictly necessary, but if it is not added there will be a zeroline at y = 0

val xaxis2 = XAxis(2) setTitle "x2"
val yaxis2 = YAxis(2) // this is not strictly necessary, but if it is not added there will be a zeroline at y = 0

val series1 = (
    XY(x = List(1, 2, 3), y = List(2, 4, 5))
    asType SCATTER
    drawStyle MARKERS
)

val series2 = (
    XY(x = x, y = y) 
    asType SCATTER 
    drawStyle MARKERS 
    setAxes (xaxis2, yaxis2)
)

val chart1 = Chart()
            .addSeries(series1, series2)
            .setTitle("Chart.Axis.Composition")
            .addAxes(xaxis1, yaxis1, xaxis2, yaxis2)
            .setConfig(responsive=false)
            // This tells Picta that we want to actually subdivide this chart into multiple plots
            .asMultiChart(1, 2)

chart1.plotInline

### Picta's Subplot System

While the above is useful, sometimes it is more useful to plot independent charts in a subplot grid. This is where we can use the Picta subplot system.

In the subplots below, each subplot is an individual `Chart`. This means we can actually embed the above `MultiChart` inside another subplot. This way we can create nested subplots to showcase any data we need to.

Used creatively, this subplot system can be very useful for data exploration.

In [15]:
// first define the x-axes we will use in the plot
val ax1 = XAxis(title = "x axis 1")
val ax2 = XAxis(title = "x axis 2")
val ax3 = XAxis(title = "x axis 3")
val ax4 = XAxis(title = "x axis 4")

// first define the y-axes we will use in the plot
val ax6 = YAxis(title = "y axis 1")
val ax7 = YAxis(title = "y axis 2")
val ax8 = YAxis(title = "y axis 3")
val ax9 = YAxis(title = "y axis 4")

// it may be necessary to play around with the chart dimensions and margin  in order to ensure a good fit on screen.
val dim = 350

val chart1 = (
    Chart()
    setDimensions(width = dim, height = dim)
    addSeries XY(x, y).setName("a")
    addAxes(ax1, ax6)
    setMargin(l=50, r=30, t=50, b=50)
)

val chart2 = (
    Chart() 
    setDimensions(width = dim, height = dim)
    addSeries XY(x, y).setName("b")
    addAxes(ax2, ax7)
    setMargin(l=50, r=50, t=50, b=50)
)

val chart3 = (
    Chart()
    setDimensions(width = dim, height = dim)
    addSeries XY(x, y).setName("c") 
    addAxes(ax3, ax8)
    setMargin(l=50, r=30, t=50, b=50)
)

val chart4 = (
    Chart()
    setDimensions(width = dim, height = dim)
    addSeries(series1, series2)
    setTitle "The MultiChart from Above"
    addAxes(xaxis1, yaxis1, xaxis2, yaxis2) // these axes come from above
    setConfig(responsive=false)
    asMultiChart(1, 2)
)

// The canvas has an underlying grid. By default the underlying grid is 1x1, but we can pass in the dimensions we 
// require by passing in parameters in the constructor.
Canvas(2, 2, title="The Picta Grid")
.setChart(0, 0, chart1)
.setChart(0, 1, chart2)
.setChart(1, 0, chart3)
.setChart(1, 1, chart4)
.plotInline

The Picta Grid 
 
 
 
 
 
 
 

 
 Download Canvas as PNG

### Error Bars

We can also add Error Bars to our plots.

Error bars come in two flavours:

1. `XError`
2. `YError`

Both of these has an associated `mode` that determines how the error bar is calculated for an individual point. The following `mode`s can be specified:

- `DATA`: The user passes in an array that specifies the per point error
- `PERCENT`: The user passes in a Double that calculates the error as a percentage of the value of the point
- `CONSTANT`: The user passes in a double that is the constant value for the error for all points
- `SQRT`: This calculates the error as a square root of the point value.

The next few examples will demonstrate:

In [16]:
// First import the Error Bar options
import org.carbonateresearch.picta.options.{YError, CONSTANT, DATA, PERCENT, SQRT}

In [17]:
val dim = 400

val series1 = (
    XY(List(1, 2, 3), List(1.234, 5.2112, 2.44332))
    asType SCATTER
    drawStyle MARKERS
    // User specifies the error per point 
    setErrorBars YError(mode = DATA, array = List(0.5, 0.5, 0.5))
)

val chart1 = (
    Chart()
    addSeries series1
    setTitle "Per Point Specified Error"
    setDimensions(width = dim, height = dim)
)


val series2 = (
    XY(List(1, 2, 3), List(1.234, 5.2112, 2.44332))
    asType SCATTER
    drawStyle MARKERS
    // the error here is 10% of the corresponding y-value for the point
    setErrorBars YError(mode = PERCENT, value = 10.0)
)

val chart2 = (
    Chart()
    addSeries series2
    setTitle "Percentage Error"
    setDimensions(width = dim, height = dim)
)


val series3 = (
    XY(List(1, 2, 3), List(1.234, 5.2112, 2.44332))
    asType SCATTER
    drawStyle MARKERS
    // a constant error of 10 is applied to each point
    setErrorBars YError(mode = CONSTANT, value = 10.0)
)

val chart3 = (
    Chart()
    addSeries series3
    setTitle "Constant Error"
    setDimensions(width = dim, height = dim)
)


val series4 = (
    XY(List(1, 2, 3), List(1.234, 5.2112, 2.44332))
    asType SCATTER
    drawStyle MARKERS
    setErrorBars YError(mode = SQRT)
)

val chart4 = (
    Chart()
    addSeries series4
    setTitle "Sqrt Error"
    // the error is set to the sqrt of the corresponding point
    setDimensions(width = dim, height = dim)
)

val canvas = Canvas(2, 2, "Errors") addCharts (chart1, chart2, chart3, chart4)

canvas.plotInline

Errors 
 
 
 
 
 
 
 

 
 Download Canvas as PNG

### Other Types of Charts

The next examples will demonstrate how to create a variety of `Chart`s. All of the examples below are composable with the `Canvas` subplot system.

#### Pie Chart

Pie charts can be created in two ways. The first way uses the `PieElement` component to compose a Piechart:

In [18]:
val a = PieElement(value=60, name="Residential")
val b = PieElement(value=20, name="Non-Residential")
val c = PieElement(value=20, name="Utility")

// we add a list of Pie Elements to an XY series as the list of Pie Elements gets deconstructed down into two series:
// X => values: [60, 20, 20]
// Y => labels: ["Residential", "Non-Residential", "Utility"]
// The labels become the series_name for each individual PieElement
// As we pass in a list of PieElements, we do not need to specify the type as a PIE
val series = XY(x=List(a, b, c))

val chart = Chart() addSeries series setTitle "Pie Elements"

chart.plotInline

In [19]:
// However, composing individual PieElements may be tedious if there is a lot of data and we know how to it all fits
// together. Picta provides a short hand to quickly create a Pie Chart using the methods we have seen previously
// In this case we pass the (value, name) pairs as two seperate lists. As before, the labels become an individual
// series name for each point; which is why the legend renders correctly.
val series = XY(x=List(60, 20, 20), y=List("Residential", "Non-Residential", "Utility")) asType PIE

val chart = Chart() addSeries series setConfig(false, false) setTitle "Pie Chart"

chart.plotInline

### Histogram

In [20]:
val series = XY(x=x) asType HISTOGRAM

val xaxis = XAxis(title = "my x data")
val yaxis = YAxis(title = "my y data")

val chart = (
    Chart() 
    addSeries series
    setTitle "Histogram with axes"
    addAxes(xaxis, yaxis)
)

Canvas()
.addCharts(chart)
.plotInline

#### Customizing colors for a Histogram

We can use the `HistOptions` class to further specify options for a histogram. For example, if we wanted to create a horizontally positioned histogram, we can do the following:

In [21]:
import org.carbonateresearch.picta.options.histogram.HistOptions
import org.carbonateresearch.picta.options.Line
import org.carbonateresearch.picta.options.HORIZONTAL

In [22]:
// we can also compose customizations in much the same way:
val marker = Marker() setColor "rgba(255, 100, 102, 0.5)" setLine Line()

// change xkey to y to get a horizontal histogram
val series = (
    XY(x) 
    asType HISTOGRAM
    setMarker marker
    // we can set histogram specific options using the setHistOptions method
    setHistOptions(orientation = HORIZONTAL)
)

val chart = (
    Chart() 
    addSeries series 
    setTitle "XY.Histogram.Color"
)

chart.plotInline

#### Cumulative Histogram

Sometimes a cumulative histogram may be desired. We can do this as follows:

In [23]:
// we can import a range of histnorms
import org.carbonateresearch.picta.options.histogram.{Cumulative, PERCENT, DENSITY, PROBABILITY_DENSITY, NUMBER}

In [24]:
val series = XY(x) asType HISTOGRAM setHistOptions(histnorm = NUMBER, cumulative = Cumulative(enabled=true))

val chart = Chart() addSeries series setTitle "Histogram - Cumulative"

chart.plotInline

#### Specifiying the Binning Function

We can also specify the binning function for a histogram.

In [25]:
import org.carbonateresearch.picta.options.histogram.{COUNT, SUM, AVG, MIN, MAX, HistOptions}

In [26]:
val x = List("Apples", "Apples", "Apples", "Oranges", "Bananas")
val y = List("5", "10", "3", "10", "5")

// we can also assign histOptions to a value and pass them to the setHistOptions method
val ho1 = HistOptions(histfunc = COUNT)
val ho2 = HistOptions(histfunc = SUM)
    
val t1 = XY(x = x, y = y) asType HISTOGRAM setHistOptions ho1
val t2 = XY(x = x, y = y) asType HISTOGRAM setHistOptions ho2

val chart = Chart() addSeries(t1, t2) setTitle "Histogram - Specify Binning Function"
val canvas = Canvas() addCharts chart
canvas.plotInline

### 2D Histogram Contour

In [27]:
val x = List.range(1, 50)
val y = x.map(x => x + scala.util.Random.nextDouble()*100)

val series = XY(x, y).asType(HISTOGRAM2DCONTOUR).drawMarkers

val chart = (
    Chart() addSeries series
    setTitle "2D Histogram Contour"
)

chart.plotInline

#### Adding additional axes

As the above is a density plot, adding histograms can be useful too.

We can add histograms as follows:

In [28]:
// import the 2d Density histogram options
import org.carbonateresearch.picta.options.histogram2d.Hist2dOptions
import org.carbonateresearch.picta.options.HORIZONTAL

In [29]:
val marker = Marker() setColor "rgb(102,0,0)"

val series1 = XY(x, y) 
            .asType(SCATTER)
            .drawMarkers
            .setName("points")
            .setMarker(marker)


val hist2d_options = Hist2dOptions(ncontours = 20, reversescale = false, showscale = true)

val series2 = XY(x, y) 
            .setName("density")
            .asType(HISTOGRAM2DCONTOUR)
            .setHist2dOptions(hist2d_options)

val series3 = XY(x = x)
            .asType(HISTOGRAM)
            .setName("histogram")
            .setAxes(XAxis(), YAxis(2))

val series4 = (XY(y)
            .setName("y density")
            .asType(HISTOGRAM)
            .setAxis(XAxis(2))
            .setMarker(marker)
            .setHistOptions(HistOptions(orientation = HORIZONTAL)))

val ax1 = XAxis(showgrid = false) setDomain(0.0, 0.85)
val ax2 = YAxis(showgrid = false) setDomain(0.0, 0.85)
val ax3 = XAxis(position = 2, showgrid = false) setDomain(0.85, 1.0)
val ax4 = YAxis(position = 2, showgrid = false) setDomain(0.85, 1.0)

val layout = (
    ChartLayout("Histogram2dContour With Density", autosize = false, showlegend = false) 
    setAxes(ax1, ax2, ax3, ax4)
)

val chart = Chart() addSeries(series1, series2, series3, series4) setChartLayout layout

chart.plotInline

### 3D Charts

3D charts are constructed using a `XYZ` series as they take in 3 `Series`' to create a single point.

The Picta API does not accept nested lists, however the underlying Plotlyjs render does. In order to render any series that will make use of a nested list, we must flatten and provide the length of an element (before the list was flattened).

The following examples will make it clearer.

#### Contour

In [30]:
// lets create some dummy adata for the third dimension
val x = List(-9, -6, -5 , -3, -1)
val y = List(0, 1, 4, 5, 7)
val z = List(
    List(10, 10.625, 12.5, 15.625, 20),
    List(5.625, 6.25, 8.125, 11.25, 15.625),
    List(2.5, 3.125, 5.0, 8.125, 12.5),
    List(0.625, 1.25, 3.125, 6.25, 10.625),
    List(0, 0.625, 2.5, 5.625, 10)
)

In [31]:
// we flatten the nested list as we pass it into the Series constructor
val series = XYZ(x=x, y=y, z=z.flatten, n=z(0).length).asType(CONTOUR)

// set up the chart
val chart = Chart()
            .addSeries(series)
            .setTitle("Contour")

// plot the chart
chart.plotInline

#### Heatmap

In [32]:
// create a new nested list for the heatmap
val z = List.range(1, 101).map(e => e + scala.util.Random.nextDouble()*100).grouped(10).toList

In [33]:
// we get the length of an element of the nested list
val n = z(0).length

// we now flatten the list and pass it into Series constructor, as well as 'n', the length of an element so that the 
// heatmap dimensions are correctly constructed
val series = XYZ(z=z.flatten, n=n) asType HEATMAP

val chart = Chart() addSeries series setTitle "Heatmap"

chart.plotInline

#### Scatter3D

In [34]:
val x = List.range(1, 100)
val y = List.range(1, 100)
val z = List.range(1, 100).map(e => e + scala.util.Random.nextDouble()*100)

val series = XYZ(x, y, z) asType SCATTER3D drawStyle MARKERS
val chart1 = Chart() addSeries series setTitle "XYZ.Scatter3D" setConfig(false, false)
val canvas = Canvas() addCharts chart1
canvas.plotInline

#### Surface Plot

In [35]:
// 3d surface plot
val k = List(
    List(8.83,8.89,8.81,8.87,8.9,8.87),
    List(8.89,8.94,8.85,8.94,8.96,8.92),
    List(8.84,8.9,8.82,8.92,8.93,8.91),
    List(8.79,8.85,8.79,8.9,8.94,8.92),
    List(8.79,8.88,8.81,8.9,8.95,8.92),
    List(8.8,8.82,8.78,8.91,8.94,8.92),
    List(8.75,8.78,8.77,8.91,8.95,8.92),
    List(8.8,8.8,8.77,8.91,8.95,8.94),
    List(8.74,8.81,8.76,8.93,8.98,8.99),
    List(8.89,8.99,8.92,9.1,9.13,9.11),
    List(8.97,8.97,8.91,9.09,9.11,9.11),
    List(9.04,9.08,9.05,9.25,9.28,9.27),
    List(9,9.01,9,9.2,9.23,9.2),
    List(8.99,8.99,8.98,9.18,9.2,9.19),
    List(8.93,8.97,8.97,9.18,9.2,9.18)
  )

val series = XYZ(z=k.flatten, n = k(0).length) asType SURFACE
val chart2 = Chart() addSeries series setTitle "Surface"

chart2.plotInline

#### Third Dimension as Color

In [36]:
// multiple compositions can be used to create scatter charts with a color representing some third dimension
val marker = Marker() setColor z
val series = XY(x, y) asType SCATTER drawStyle MARKERS setMarker marker
val chart3 = Chart() addSeries series setTitle "ScatterWithColor"
chart3.plotInline

#### Subplot

The Subplot class can be used to generate subplots for an XYZ plots just as we did previously.

In [37]:
// it may be necessary to play around with the chart dimensions and margin  in order to ensure a good fit on screen.
val dim = 350

// The canvas has an underlying grid. By default the underlying grid is 1x1, but we can pass in the dimensions we 
// require by passing in parameters in the constructor.
Canvas(2, 2)
.setChart(0, 0, chart1.setDimensions(width = dim, height = dim).setMargin(l=50, r=50, t=50, b=50))
.setChart(0, 1, chart2.setDimensions(width = dim, height = dim)setMargin(l=50, r=50, t=50, b=50))
.setChart(1, 0, chart3.setDimensions(width = dim, height = dim)setMargin(l=50, r=50, t=50, b=50))
.setChart(1, 1, chart.setDimensions(width = dim, height = dim)setMargin(l=50, r=50, t=50, b=50))
.plotInline

Download Canvas as PNG

#### Map

We can also create maps using the composition technique below.

In [38]:
import org.carbonateresearch.picta.options.{Margin, Line}
import org.carbonateresearch.picta.Map

In [39]:
// draw a line on the map that is red
val line = Line(width = 2) setColor "red"

// construct the map Series. It is an XY chart as it takes in a List of Longitude and Latitude
val series = Map(List(40.7127, 51.5072), List(-74.0059, 0.1275)) drawSymbol LINES drawLine line

// These are options that further specify the options for the map
val geo = MapOptions(landcolor = "rgb(204, 204, 204)", lakecolor = "rgb(255, 255, 255)")
              .setMapAxes(LatAxis(List(20, 60)), LongAxis(List(-100, 20)))

val chart = (
    Chart() 
    addSeries series 
    setConfig(false, false)
    setMapOptions geo
    setMargin(l=0, r=0, t=0, b=0)
)

chart.plotInline

### Animated Charts

We can also create animated charts. This can be useful for tracking the evolution of a data over time.

All `Series` types should be supported.

#### 2D Animated Chart

In [40]:
import org.carbonateresearch.picta.options.ColorOptions._

// creates random XY for testing purposes
def createXYSeries[T: Color]
(numberToCreate: Int, count: Int = 0, length: Int = 10): List[XY[Int, Double, T, T]] = {
    if (count == numberToCreate) Nil
    else {
      val xs = List.range(0, length)
      val ys = xs.map(x => scala.util.Random.nextDouble() * x)
      val series = XY(x = xs, y = ys, name = "series " + count).drawMarkers
      series :: createXYSeries(numberToCreate, count + 1, length)
    }
}

def createXYZSeries(numberToCreate: Int, count: Int = 0, length: Int = 10): List[XYZ[Int, Double, Double]] = {
    if (count == numberToCreate) Nil
    else {
      val xs = List.range(0, length)
      val ys = xs.map(x => scala.util.Random.nextDouble() * x)
      val zs = xs.map(x => scala.util.Random.nextDouble() * x * scala.util.Random.nextInt())
      val series = XYZ(x = xs, y = ys, z = zs, name = "series " + count, `type` = SCATTER3D).drawMarkers
      series :: createXYZSeries(numberToCreate, count + 1, length)
    }
}

In [41]:
val xaxis = XAxis(title = "X Variable") setRange (0.0, 10.0)
val yaxis = YAxis(title = "Y Variable") setRange (0.0, 10.0)

// we can also specifiy the underlying layout directly - sometimes this can be useful
val layout = ChartLayout("Animation XY") setAxes(xaxis, yaxis)

val series = createXYSeries(numberToCreate = 50, length = 30)

val chart = Chart(animated = true, transition_duration=100) setChartLayout layout addSeries series

chart.plotInline

Play 
 Pause 
 
 
 Frame: 0

### 3D Animated Chart

In [42]:
val series = createXYZSeries(numberToCreate = 10, length = 100)

val layout = ChartLayout()

val chart = Chart(animated = true) setTitle "Animation 3D" addSeries series 

chart.plotInline

Play 
 Pause 
 
 
 Frame: 0

## IO + Utility Functions

The library also comes with some basic CSV IO functions and a utility function for breaking down data

In [43]:
import org.carbonateresearch.picta.IO._
import org.carbonateresearch.picta.common.Utils.getSeriesbyCategory

In [44]:
val working_directory = getWorkingDirectory

println(working_directory)

/Users/fazi/Desktop/Final Project/irp-acse-fk4517/Code


In [45]:
// by providing a path, we can read in a CSV
val filepath = working_directory + "/iris_csv.csv"

val data = readCSV(filepath)

// by default, CSV are read in as strings. However we can convert the individual columns to the correct format
val sepal_length = data("sepallength").map(_.toDouble)
val petal_width = data("petalwidth").map(_.toDouble)
val categories = data("class")

In [46]:
val series = XY(sepal_length, petal_width) asType SCATTER drawStyle MARKERS

val chart = Chart() addSeries series setTitle "Uninformative Chart"

chart.plotInline

The above chart is not very informative. However since we have the per data point category labels, we can use the utility function to display the data in the different clusters

In [47]:
val result = getSeriesbyCategory(categories, (sepal_length, petal_width))

val chart = Chart() addSeries result setTitle "Iris" showLegend true

chart.plotInline

## Conus Integration

A couple of utility functions make wrangling conus data easier. Below is an example taken from the conus repo and plotting some variables of interest.

In [48]:
import org.carbonateresearch.picta.conus.Utils._

In [49]:
import org.carbonateresearch.picta.conus.Utils._
import org.carbonateresearch.picta.{Canvas, Chart, XY}
import org.carbonateresearch.conus.common.SingleModelResults
import org.carbonateresearch.conus.modelzoo.GeneralGeology._
import org.carbonateresearch.conus.modelzoo.PasseyHenkesClumpedDiffusionModel._
import org.carbonateresearch.conus.{AllCells, ModelVariable, PerCell, Simulator, Step, SteppedModel}

import math._

In [50]:
val modelWarehouse = Simulator

// a few constants
val rhocal:Double = 2.71 //Density of carbonates
val cOf=889000 //concentration of O in fluid
val cCf=200 //concentration of C in fluid
val cOcal = 480000 //concentration of O in stoichiometric calcite
val cCcal = 120000 //concentration of C in stoichiometric calcite
val d13Ctdc = 0 //carbon isotope composition of total dissolved carbon in fluid

// setting model variables
val initialAge:ModelVariable[Double] = ModelVariable("Initial age",96.0,"Ma")
val finalAge:ModelVariable[Double] = ModelVariable("Final age",0.0,"Ma")
val d18Of:ModelVariable[Double] = ModelVariable("Fluid d18O",-1.0,"‰")
val TmaxSample:ModelVariable[Double] = ModelVariable("Tmax",0.0,"˚C")
val WFF:ModelVariable[Double] = ModelVariable("Weight fraction of fluid",0.0,"")
val CWRR:ModelVariable[Double] = ModelVariable("Cumulative water/rock ratio",0.0,"")
val FluidMass:ModelVariable[Double] = ModelVariable("Mass of fluid",0.2,"")
val d18Occ:ModelVariable[Double] = ModelVariable("d18Occ",1.0,"‰")
val d13Ccc:ModelVariable[Double] = ModelVariable("d13Ccc",1.0,"‰")
val D47r:ModelVariable[Double] = ModelVariable("D47 with partial recrystallization",.5,"‰")
val fractionRec:ModelVariable[Double] = ModelVariable("Fraction of recrystallization",0.01,"‰")
val initalBurialAtModelStart:ModelVariable[Double] = ModelVariable("Initial burial at model start",0.0,"meters")

// Initialise model conditions as lists
val burialHistory = List((96.0,0.0), (39.0,2000.0), (0.0,-70.0))
val geothermalGradientHistory= List((96.0,30.0),(0.0,30.0))
val surfaceTemperaturesHistory = List((96.0,30.0),(0.0,30.0))
val numberOfSteps = 10
val ageList:List[Double] = List(97.0,96.0,95.0)
val finalAgeList:List[Double] = List(0.0)
val rangeOfFluidMasses:List[Double] = List(0.2)
val rangeOfPartialRecrystallization = List(1.0,.5,.1,.01,0.0)

// Bulk isotope calculation equations
val d18OccFunction = (s:Step) => {
    val cO=(WFF(s)*cOf)+(1-WFF(s))*cOcal
    val alpha = math.exp((18.03*(1000*pow((burialTemperature(s)+273.15), -1)) - 32.42)/1000)
    val d18O = ((d18Of(s) * WFF(s) * cOf) + d18Occ(s) * (1-WFF(s)) * cOf)/ cO
    ((d18O * cOf * alpha) - (1000 * cOf * WFF(s) * (1-alpha))) / ((cOcal * (1-WFF(s)) * alpha) + cOf * WFF(s))
}

val d13CccFunction = (s:Step) => {
    val cC=(WFF(s)*cCf)+(1-WFF(s))*cCcal
    val alpha = exp((-2.4612+(7666.3/100) - (2.9880*pow(10,3)/pow((burialTemperature(s)+273.15),6)))/1000)
    val d13C = ((d13Ctdc * WFF(s) * d13Ctdc) + (d13Ccc(s) * (1-WFF(s)) * cCcal))/cC
    ((d13C * cC * alpha) - (1000 * cCf * WFF(s) * (1-alpha))) / (cCcal * (1-WFF(s) * alpha) + cCf * WFF(s))
}

val eaglefordModel = new SteppedModel(numberOfSteps,"Eagleford recrystallization")
    .setGrid(19)
    .defineMathematicalModel(
      age =>> ageOfStep(initialAge,finalAge),
      depth =>> {(s:Step) => {burialDepthFromAgeModel(age,burialHistory).apply(s)+initalBurialAtModelStart(s)}},
      surfaceTemperature =>> surfaceTemperaturesAtAge(age, surfaceTemperaturesHistory),
      geothermalGradient =>> geothermalGradientAtAge(age,geothermalGradientHistory),
      burialTemperature =>> burialTemperatureFromGeothermalGradient(surfaceTemperature,depth,geothermalGradient),
      dT =>> dTFun,
      D47eq =>> D47eqFun,
      D47i =>> D47iFun,
      SampleTemp =>> davies19_T,
      FluidMass =>> {(s:Step) => 0.2},
      d18Of =>> {(s:Step) => -1.0},
      WFF =>> {(s:Step) => FluidMass(s) / (FluidMass(s) + rhocal)},
      CWRR =>> {(s:Step) => s.stepNumber * WFF(s)/(1-WFF(s))},
      d18Occ =>> d18OccFunction,
      d13Ccc =>> d13CccFunction,
      D47r =>> {(s:Step) => {if(burialTemperature(s)-burialTemperature(s-1) >= 0){D47r(s-1) * (1.0-fractionRec(s)) + D47eq(s) * fractionRec(s)}
      else {D47r(s-1)}}}
    )
    .defineInitialModelConditions(
      AllCells(initialAge,ageList),
      AllCells(finalAge,finalAgeList),
      AllCells(fractionRec,rangeOfPartialRecrystallization),
      AllCells(D47i,List(0.670,0.680,0.690)),
      AllCells(D47r,List(0.670,0.680,0.690)),
      PerCell(initalBurialAtModelStart,List(
        (List(0.0),Seq(0)),
        (List(-0.91),Seq(1)),
        (List(-1.22),Seq(2)),
        (List(-2.90),Seq(3)),
        (List(-4.57),Seq(4)),
        (List(-4.88),Seq(5)),
        (List(-4.90),Seq(6)),
        (List(-6.10),Seq(7)),
        (List(-6.40),Seq(8)),
        (List(-6.71),Seq(9)),
        (List(-25.90),Seq(10)),
        (List(-28.96),Seq(11)),
        (List(-29.87),Seq(12)),
        (List(-35.05),Seq(13)),
        (List(-35.36),Seq(14)),
        (List(-35.97),Seq(15)),
        (List(-36.58),Seq(16)),
        (List(-37.19),Seq(17)),
        (List(-38.90),Seq(18)))))
    .defineCalibration(
      D47r.isBetween(0.511,0.683).atCells(Seq(0)),
      D47r.isBetween(0.511,0.683).atCells(Seq(1)),
      D47r.isBetween(0.511,0.683).atCells(Seq(2)),
      D47r.isBetween(0.511,0.683).atCells(Seq(3)),
      D47r.isBetween(0.511,0.683).atCells(Seq(4)),
      D47r.isBetween(0.511,0.683).atCells(Seq(5)),
      D47r.isBetween(0.511,0.683).atCells(Seq(6)),
      D47r.isBetween(0.511,0.683).atCells(Seq(7)),
      D47r.isBetween(0.511,0.683).atCells(Seq(8)),
      D47r.isBetween(0.511,0.683).atCells(Seq(9)),
      D47r.isBetween(0.511,0.683).atCells(Seq(10)),
      D47r.isBetween(0.511,0.683).atCells(Seq(11)),
      D47r.isBetween(0.511,0.683).atCells(Seq(12)),
      D47r.isBetween(0.511,0.683).atCells(Seq(13)),
      D47r.isBetween(0.511,0.683).atCells(Seq(14)),
      D47r.isBetween(0.511,0.683).atCells(Seq(15)),
      D47r.isBetween(0.511,0.683).atCells(Seq(16)),
      D47r.isBetween(0.511,0.683).atCells(Seq(17)),
      D47r.isBetween(0.511,0.683).atCells(Seq(18))
)

modelWarehouse.evaluate(eaglefordModel)
Thread.sleep(1000)

15:36:10.591 [CoNuS-akka.actor.default-dispatcher-5] INFO akka.event.slf4j.Slf4jLogger - Slf4jLogger started
A total of 135 unique models were defined, attempting to create a list now.
Models list successfully created.
----------------------------------------
RUN STARTED
----------------------------------------
RUN DATA
Total number of models: 135
Number of cell per grid: 19
Number of steps per model: 10
Number of operation per step: 304
Total number of operations: 410'400
Available CPU cores: 8
----------------------------------------
RUN PROGRESS
Model #4 completed in 0 seconds is calibrated.
Model #6 completed in 0 seconds is calibrated.
Model #7 completed in 0 seconds is calibrated.
Model #5 completed in 0 seconds is calibrated.
Model #1 completed in 0 seconds is calibrated.
Model #2 completed in 0 seconds is calibrated.
Model #8 completed in 0 seconds is calibrated.
Model #9 completed in 0 seconds is calibrated.
Model #10 completed in 0 seconds is calibrated.
Model #12 completed i

In [51]:
// grab the results from the Conus model
val model: SingleModelResults = modelWarehouse(eaglefordModel)(0)

// we can use the utility function to grab the series for a single variable
val series1: List[Double] = getDataFromSingleModel(model, age, List(0), numberOfSteps)
val series2: List[Double] = getDataFromSingleModel(model, d18Occ, List(0), numberOfSteps)

val xy1 = XY(series1, series2)

// alternatively we can quickly get the same data for XY using the function below
//  val xy1 = getXYSeriesFromSingleModel(model, (age, d18Occ), List(0), numberOfSteps)

// lets also plot a second y variable
val yaxis2 = YAxis(
    position = 2, 
    title = "d13Ccc", 
    overlaying = YAxis(), // this ensures that the axis sits on a seperate axis
    side = RIGHT, // this ensures the axis is on the right hand side
    tickformat = "0.2f" // this will keep formatting reasonable for display purposes
)

// we construct the second y variable;
val series3: List[Double] = getDataFromSingleModel(model, d13Ccc, List(0), numberOfSteps)
val xy2 = XY(series1, series3) setAxis yaxis2

// finally we can combine in a single chart
val chart = (
    Chart() 
    addSeries xy1
    addSeries xy2
    setTitle("d18Occ vs d13Ccc") 
    addAxes(XAxis(title="age"), YAxis(title="d18Occ"), yaxis2)
)

val canvas = Canvas() addCharts chart

// When we plot the result, we can see the legend is in the wrong place and overlaying the axis - we can overcome this
// in the next example
canvas.plotInline()

### Moving the Legend

We can also move the Legend and position it where we would like as follows:

In [52]:
import org.carbonateresearch.picta.options.AUTO
import org.carbonateresearch.picta.options.VERTICAL

In [55]:

// finally we can combine in a single chart
val chart = (
    Chart() 
    addSeries xy1
    setTitle("Move Axis") 
    addAxes(XAxis(title="age"), YAxis(title="d18Occ"), yaxis2)
    setLegend(x = 0.5, y = -0.5, orientation = HORIZONTAL, xanchor = AUTO, yanchor = AUTO)
)

val canvas = Canvas() addCharts chart

canvas.plotInline()